In [5]:
# Import Library
import pandas as pd
import numpy as np
from keras.models import load_model
import tensorflow as tf
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

In [26]:
# Import Dataset
import pandas as pd
path = 'dataset/'
firm_df = pd.read_pickle(path+'firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle(path+'macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#macro_scaled = scaler.fit_transform(macro_new)
#macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
#macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_new, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [27]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

## Model 5a: 49 Firm-level + 8 Macroeconomic Variable

In [28]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro8")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 57) (88347,) (31851, 57) (31851,) (90007, 57) (90007,)


In [29]:
# Linear Model Training 
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
regr = linear_model.LinearRegression()
regr.fit(x_all, y_all)

# Print out Predictive R^2
y_predict = regr.predict(X_test)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

yhat_df = pd.DataFrame()
yhat_df['OLSa_49_8'] = y_predict
path_y = 'predict/'
yhat_df.to_pickle(path_y+"model5a49_8.pkl")

R^2 is -0.03882505278646131


/Users/scarlett_yin/Documents/Upgrade_paper/.venv/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


## Model 5b: 49 Firm-level + 134 Macroeconomic Variable 

In [30]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 183) (88347,) (31851, 183) (31851,) (90007, 183) (90007,)


In [31]:
# Linear Model Training 
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
regr = linear_model.LinearRegression()
regr.fit(x_all, y_all)

# Print out Predictive R^2
y_predict = regr.predict(X_test)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

yhat_df = pd.DataFrame()
yhat_df['OLSa_49_134'] = y_predict
path_y = 'predict/'
yhat_df.to_pickle(path_y+"model5b49_134.pkl")

R^2 is -31.322320470978866


/Users/scarlett_yin/Documents/Upgrade_paper/.venv/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


## Model 5c: 49 Firm-level

In [32]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(firm_df, dataset = "firm")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 49) (88347,) (31851, 49) (31851,) (90007, 49) (90007,)


In [33]:
# Linear Model Training 
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
regr = linear_model.LinearRegression()
regr.fit(x_all, y_all)

# Print out Predictive R^2
y_predict = regr.predict(X_test)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

yhat_df = pd.DataFrame()
yhat_df['OLSa_49'] = y_predict
path_y = 'predict/'
yhat_df.to_pickle(path_y+"model5c49.pkl")

R^2 is 0.005432492063096261


/Users/scarlett_yin/Documents/Upgrade_paper/.venv/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


## Finished 

X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)